In [38]:
import pandas as pd
import numpy as np
import json
import csv
from pandasql import sqldf
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib as plt
import seaborn as sns
%matplotlib inline

pysqldf = lambda q: sqldf(q, globals())


In [4]:

with open("data/song.json") as file:
    data = json.load(file)
    
    

In [9]:
#convert json to csv
with open("data/song.csv", "w") as file:
    csv_file = csv.writer(file)
    csv_file.writerow(["id","song_played","time_played","user_id","user_sign_up_date","user_state"])
    for item in data:
        csv_file.writerow([item['id'], item['song_played'],item['time_played'],item['user_id'],item['user_sign_up_date'],item['user_state'] ])

In [10]:
del data



In [99]:
df = pd.read_csv("data/song.csv")

In [100]:
df[["time_played","user_sign_up_date"]] = df[["time_played","user_sign_up_date"]].apply(pd.to_datetime)

In [101]:
df.head()

,id,song_played,time_played,user_id,user_sign_up_date,user_state
0,GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
1,HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio
2,DKQSXVNJDH,Back In the U.S.S.R.,2015-06-14 02:11:29,35,2015-05-04,New Jersey
3,HLHRIDQTUW,P.s. I Love You,2015-06-08 12:26:10,126,2015-05-16,Illinois
4,SUKJCSBCYW,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,6,2015-05-01,New Jersey


In [103]:
dfdf.iloc[0,1]

Timestamp('2015-05-16 00:00:00')

In [104]:
df.isnull().sum()

id                   0
song_played          0
time_played          0
user_id              0
user_sign_up_date    0
user_state           0
dtype: int64

let's find count of unique users per state. 

In [105]:
unique_users_per_state = df.drop_duplicates(subset=["user_id","user_state"])[["user_id","user_state"]]

In [106]:
unique_users_per_state.groupby("user_state").count().sort_values("user_id",axis=0,ascending=False)

,user_id
user_state,
New York,23
California,21
Texas,15
Pennsylvania,9
Ohio,9
Illinois,7
Florida,7
North Carolina,6
New Jersey,6


In [107]:
early_birds = df.drop_duplicates(subset=["user_id","user_state"]).sort_values('user_sign_up_date',axis=0,ascending=True)
early_birds.groupby('user_state')[["user_id","user_sign_up_date"]].first()

,user_id,user_sign_up_date
user_state,,
Alabama,5,2015-05-01
Alaska,106,2015-05-12
Arizona,105,2015-05-12
Arkansas,78,2015-05-08
California,39,2015-05-04
Colorado,173,2015-05-19
Connecticut,127,2015-05-16
Florida,41,2015-05-04
Georgia,16,2015-05-02


Unique users

In [49]:
print pysqldf("select count(user_id) from df")

   count(user_id)
0            4000


In [60]:
#print pysqldf("select distinct user_id, user_sign_up_date, user_state from df group by user_state order by user_sign_up_date")

### Recommendation

Here we build a function that takes as an input any of the songs in the data and returns the most likely song to be listened next. That is, if, for instance, a user is currently listening to "Eight Days A Week", which song has the highest probability of being played right after it by the same user? This is going to be v1 of a song recommendation model.

In [212]:
sorted_play_times = df.sort_values("time_played",axis=0)

query = "Come Together"
groups = sorted_play_times.groupby("user_id")

next_tracks_listened = []

for name, group in groups:
    qindex = group.song_played == query
    first_play = group[qindex].index.tolist()
    
    
    if not first_play: #skip if user not listened to the track yet
        continue 
    
#     print qindex
#     print qindex.shift()
    
    next_track_index = qindex.shift()
    next_track_index.iloc[0] = False #avoid NaN index that happens after shifting
    
    #ToDo: drop the track if time difference between the consequetive playbacks is more than x minutes
    
    next_played = group[next_track_index][["song_played"]]
    
#     print next_played
        
#     print sorted_play_times[sorted_play_times.index.isin(first_play)][["song_played","time_played","user_id"]]
#     print "--"
#     print sorted_play_times[sorted_play_times.index.isin(next_play)][["song_played","time_played","user_id"]]
#     print "xx"

    next_tracks_listened.extend(list(next_played.song_played))
    
    
# qindex = sorted_play_times[sorted_play_times.song_played == query].index.tolist()

# qnext = [x+1 for x in qindex]
# searchindex = set(qnext).union(qindex)
q_results = pd.DataFrame({"songs":next_tracks_listened , "counter":[1]*len(next_tracks_listened)}, index=np.arange(len(next_tracks_listened)))
q_results
q_results.groupby("songs").count().sort_values('counter',axis=0,ascending = False)

,counter
songs,
Come Together,43
Revolution,30
Get Back,25
Back In the U.S.S.R.,21
Let It Be,21
A Day In The Life,12
Hello Goodbye,12
Lucy In The Sky With Diamonds,12
Here Comes The Sun,11
